In [1]:
import os
import gc
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torchvision.io import read_image
import torchvision as tvis
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

/home/itsmeale/code/lab/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_path = Path("../data/xray/train/")
classes = os.listdir(root_path)

In [3]:
for c in classes:
    class_path = root_path.joinpath(c)
    c_files = [
        (c, class_path.joinpath(f))
        for f in os.listdir(class_path)
    ]

In [4]:
preprocess = tvis.transforms.Compose([
    tvis.transforms.ToTensor(),
    tvis.transforms.Resize((128, 128)),
    tvis.transforms.Grayscale(),    
])

class XRayDataset(Dataset):
    
    def __init__(self, root_dir: str):
        self.root_dir = root_dir
        self.get_dataset_files()
        self.load_dataset()
    
    def get_dataset_files(self):
        root_path = Path(self.root_dir)
        classes = os.listdir(root_path)
        
        c_files = list()
        for c in classes:
            class_path = root_path.joinpath(c)
            c_files += [
                (np.float64(classes.index(c)), class_path.joinpath(f))
                for f in os.listdir(class_path)
            ]

        self.files = c_files
    
    def load_dataset(self):
        self.images = [
            (
                preprocess(Image.open(path)),
                torch.as_tensor(c).type(torch.FloatTensor)
            )
            for c, path in self.files
        ]
        self.target = [c.item() for _, c in self.images]
    
    def load_to_gpu(self):
        if not torch.cuda.is_available():
            raise("CUDA isn't available")
        
        cuda_images = [
            (im.to("cuda"), c.to("cuda"))
            for im, c in self.images
        ]
             
        self.images = cuda_images
    
    def free(self):
        for im, c in self.images:
            im.to("cpu")
            del im
            c.to("cpu")
            del c
        del self.images
        del self.target
        gc.collect()
        torch.cuda.empty_cache()

    def __getitem__(self, idx):
        # should return a image and a label
        return self.images[idx]
    
    def __len__(self):
        # should return the number of images on the dataset
        return len(self.files)

In [5]:
dataset = XRayDataset("../data/xray/train/")

In [6]:
np.mean(dataset.target)

0.7429064417177914

---

## Modelo generativo de difusão

In [7]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion

In [8]:
model = Unet(
    dim=16,
    dim_mults=(1, 2),
    channels=1
).cuda()

diffusion = GaussianDiffusion(
    model,
    image_size=128,
    timesteps=10,
    loss_type='l1'
).cuda()

Imagens normais

In [9]:
normal = list()
for i, c in iter(dataset):
    if c.item() == 0:
        normal.append(i)

In [10]:
normal = torch.stack(normal[:100]).cuda()

In [11]:
epochs = 10

In [12]:
for e in range(epochs):
    loss = diffusion(normal)
    loss.backward()
    print(loss.item())

OutOfMemoryError: CUDA out of memory. Tried to allocate 25.00 GiB (GPU 0; 11.77 GiB total capacity; 9.32 GiB already allocated; 1.71 GiB free; 9.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

---

## Rede convolucional

In [44]:
from torch import nn

In [55]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bnfc = nn.BatchNorm1d(10)
        
        self.pool = nn.AvgPool2d(10, stride=2)

        self.fc = nn.Linear(3136, 1)
        
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
    
    def conv_layer(self, x):
        x = self.pool(self.bn1(self.conv1(x)))
        x = self.pool(self.bn2(self.conv2(x)))
        x = self.pool(self.bn3(self.conv3(x)))
        return x
    
    def fc_layer(self, x):
        x = self.sig((self.fc(x)))
        return x
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = torch.flatten(x, 1)
        x = self.fc_layer(x)
        return x


In [58]:
def train_loop(model, train_loader, optimizer, loss_fn):

    for images, labels in train_loader:
        pred = model(images)
        loss = loss_fn(pred, labels.unsqueeze(1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return loss.item()

In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [60]:
dataset.load_to_gpu()

In [61]:
model = ConvNet()
model = model.to(device)

In [63]:
epochs = 30

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=.9)

for epoch in range(epochs):
    loss = train_loop(model, train_loader, optimizer, loss_fn)
    print(f"Epoch {epoch}, loss: {loss:.4f}")

Epoch 0, loss: 0.2825
Epoch 1, loss: 0.2530
Epoch 2, loss: 0.1995
Epoch 3, loss: 0.2401
Epoch 4, loss: 0.2487
Epoch 5, loss: 0.2240
Epoch 6, loss: 0.2921
Epoch 7, loss: 0.2914
Epoch 8, loss: 0.1939
Epoch 9, loss: 0.2437
Epoch 10, loss: 0.2200
Epoch 11, loss: 0.2596
Epoch 12, loss: 0.2432
Epoch 13, loss: 0.2729
Epoch 14, loss: 0.1881
Epoch 15, loss: 0.3608
Epoch 16, loss: 0.2137
Epoch 17, loss: 0.1794
Epoch 18, loss: 0.2443
Epoch 19, loss: 0.2217
Epoch 20, loss: 0.3179
Epoch 21, loss: 0.2074
Epoch 22, loss: 0.2009
Epoch 23, loss: 0.1976
Epoch 24, loss: 0.1964
Epoch 25, loss: 0.2014
Epoch 26, loss: 0.1648
Epoch 27, loss: 0.2509
Epoch 28, loss: 0.1659
Epoch 29, loss: 0.2662
